Aqui são feitos as importações de todas as bibliotecas e ferramentas usadas em todos os exemplos.

In [111]:
from enum import auto, IntFlag
from typing import Any, Optional, Self
from datetime import datetime
from uuid import uuid4

import hashlib
import re

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    UUID4,
    field_validator,
    field_serializer,
    model_validator,
    model_serializer,
    ValidationError,
)

Na primeira classe, são definidos quais os tipos jurídicos que uma empresa pode ter. Na segunda classe, é definido os campos e as funções que uma empresa pode ter, tentei usar o máximo de tipos de dados diferentes para tratamentos distintos, como a string de email, a conta do banco sendo uma string secreta e o id sendo do tipo UUID4. Construí também um validador de campo bem simples e um validador de modelo, usando, só como exemplo, uma regra que relaciona a quantidade de sócios e o tipo jurídico da empresa. Também há os serializadores, que apenas personalizam o a transformação do objeto para JSON.

In [112]:
class juridico(IntFlag):
    ei = auto()
    ltda = auto()
    slu = auto()
    sa = auto()

class Empresa(BaseModel):
    model_config = {"extra": "forbid"}
    __empresas__ = []
    nome: str = Field(..., description="O nome da empresa", )
    id: UUID4 = Field(default_factory=uuid4, description="O identificador da empresa", frozen = True, include=True)
    tipo: juridico = Field(..., description="O tamanho jurídico da empresa", examples="LTDA")
    email: EmailStr = Field(default=None, description="O email da empresa")
    data_abertura: datetime = Field(default_factory=datetime.now, description="A data de quando a empresa foi criada")
    conta_banco: SecretStr = Field(..., description="A conta bancária da empresa", exclude=True)
    socios: list[UUID4] = Field(
        default_factory=list, max_length=500, description="lista dos sócios"
    )

    @field_validator("tipo",mode="before")
    def validar_tipo(cls, porte: int | str | juridico) -> juridico:
        op = {int: lambda x: juridico(x), str: lambda x: juridico[x], juridico: lambda x: x}
        try:
            return op[type(porte)](porte)
        except (KeyError, ValueError):
            raise ValueError(
                "Tipo da empresa é inválido"
            )

    @model_validator(mode="before")
    def validar_empresa(cls, emp: dict[str, Any]) -> dict[str, Any]:
        if emp.get("tipo") in ("sa", "SA", juridico.sa) and len(emp.get("socios", [])) < 2:
            raise ValueError("Empresa SA deve possuir pelo menos dois sócios")
        if "conta_banco" in emp:
            emp["conta_banco"] = hashlib.sha256(emp["conta_banco"].encode()).hexdigest()
        return emp

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID4) -> str:
        return str(id)

    @model_serializer(mode="wrap", when_used="json")
    def serialize_empresa(self, serializer, info) -> dict[str, Any]:
        if not info.include and not info.exclude:
            return {"nome": self.nome, "tipo": self.tipo.name, "id": self.id}
        return serializer(self)

C:\Users\Filipe\AppData\Local\Temp\ipykernel_14464\3392051929.py:11: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'include'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  id: UUID4 = Field(default_factory=uuid4, description="O identificador da empresa", frozen = True, include=True)
C:\Users\Filipe\AppData\Local\Temp\ipykernel_14464\3392051929.py:11: PydanticDeprecatedSince20: `include` is deprecated and does nothing. It will be removed, use `exclude` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  id: UUID4 = Field(default_factory=uuid4, description="O identificador da empresa", frozen = True, include=True)


Nesse bloco são definidos alguns endpoints de requisições HTTP, como já foi mostrado anteriormente os mesmos comandos. Porém dessa vez o código foi adaptado para o contexto atual. Há o comando de recuperar a lista das empresas, registrar uma empresa nessa lista, e recuperar as informações de uma empresa específica, pelo id.

In [113]:
app = FastAPI()

@app.get("/empresas", response_model=list[Empresa])
async def get_empresas() -> list[Empresa]:
    return list(Empresa.__empresas__)

@app.post("/empresas", response_model=Empresa)
async def create_empresa(empresa: Empresa):
    Empresa.__empresas__.append(empresa)
    return empresa

@app.get("/empresas/{id_empresa}", response_model=Empresa)
async def get_empresa(id_empresa: UUID4) -> Empresa | JSONResponse:
    try:
        return next((empresa for empresa in Empresa.__empresas__ if empresa.id == id_empresa))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "Empresa não encontrada"})

Para a main, foi criado um caso de teste para analisar se alguns validadores estão funcionando corretamente, e é possível ver que o model_validate percebeu que o tipo jurídico da empresa teste e a quantidade de sócios estão incoerentes (uma "SA" precisa de pelo menos dois sócios). Além disso, nos endpoints também é possível ver que ao requisitar uma empresa com id não presente na lista é retornado um erro de não encontrado (404).

In [114]:
def main():

    teste = {
        "nome": "Tech Solutions",
        "tipo": "ltda",
        "email": "contato@tech.com",
        "conta_banco": "123456789",
        "socios": [str(uuid4())]
    }

    empresa = Empresa.model_validate(teste)

    print("Objeto criado:")
    print(empresa)
    print()

    print("teste de serialização padrão (dict): ")
    print(empresa.model_dump())
    print()

    print("teste de serialização JSON:")
    print(empresa.model_dump(mode="json"))
    print()

    print("teste de erro de validação:")

    try:
        Empresa.model_validate({
            "nome": "Empresa SA",
            "tipo": "sa",
            "conta_banco": "987654321",
            "socios": [uuid4()]
        })
    except ValidationError as e:
        print("Erro capturado:")
        print(e)
        print()

    print("teste de endpoints do FastAPI")

    with TestClient(app) as client:

        response = client.post("/empresas", json=teste)
        print("POST /empresas →")
        print(response.json())
        print()

        response = client.get("/empresas")
        print("GET /empresas →")
        print(response.json())
        print()

        empresa_id = response.json()[0]["id"]
        response = client.get(f"/empresas/{empresa_id}")
        print("GET /empresas/{id}")
        print(response.json())
        print()

        response = client.get(f"/empresas/{uuid4()}")
        print("GET /empresas/f5ea767b-bf9e-4a4f-8f69-0aec02db0e09")
        print(response.json())
        print()
    
main()


Objeto criado:
nome='Tech Solutions' id=UUID('0a8b3e11-65ca-48e5-9c9b-7fe8fb34cd0a') tipo=<juridico.ltda: 2> email='contato@tech.com' data_abertura=datetime.datetime(2026, 2, 12, 21, 8, 14, 705905) conta_banco=SecretStr('**********') socios=[UUID('2ede3c8e-b1c4-4609-b596-926234d6243d')]

teste de serialização padrão (dict): 
{'nome': 'Tech Solutions', 'id': UUID('0a8b3e11-65ca-48e5-9c9b-7fe8fb34cd0a'), 'tipo': <juridico.ltda: 2>, 'email': 'contato@tech.com', 'data_abertura': datetime.datetime(2026, 2, 12, 21, 8, 14, 705905), 'socios': [UUID('2ede3c8e-b1c4-4609-b596-926234d6243d')]}

teste de serialização JSON:
{'nome': 'Tech Solutions', 'tipo': 'ltda', 'id': '0a8b3e11-65ca-48e5-9c9b-7fe8fb34cd0a'}

teste de erro de validação:
Erro capturado:
1 validation error for Empresa
  Value error, Empresa SA deve possuir pelo menos dois sócios [type=value_error, input_value={'nome': 'Empresa SA', 't...4c-9f4f-eb2bb87f088f')]}, input_type=dict]
    For further information visit https://errors.pyda